In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

ImportError: cannot import name 'LayerNormalization' from 'tensorflow.python.keras.layers.normalization' (C:\Users\Jake\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\layers\normalization\__init__.py)

In [4]:
print("Num GPUs available: {}".format(len(tf.config.experimental.list_physical_devices('GPU'))))

NameError: name 'tf' is not defined

In [17]:
# Taken from: https://keras.io/guides/sequential_model/
## Just taking a look here, NOT meant to be run
####
####
####

model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
# 32 filters, of size 5, with stride 2, activation give
model.add(layers.Conv2D(32, 5, activation="relu"))
# 32 filters again, size 3, stride 2
model.add(layers.Conv2D(32, 3, activation="relu"))
# Maxpool with 3x3 window
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 246, 246, 32)      2432      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 244, 244, 32)      9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 81, 81, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 246, 246, 32)      2432      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 244, 244, 32)      9248      
____________________________

In [5]:
# keras image classification from scratch: (See README for URL)
image_size = (256, 256)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "mel-grams",
    label_mode="categorical",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "mel-grams",
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 811 files belonging to 10 classes.
Using 649 files for training.
Found 811 files belonging to 10 classes.
Using 162 files for validation.


In [6]:
model = keras.Sequential()
# input layer
model.add(keras.Input(shape=(256, 256, 3)))


model.add(layers.experimental.preprocessing.Rescaling(1./255))
model.add(layers.Conv2D(3, 3, strides=1, activation="relu"))
model.add(layers.Conv2D(5, 3, strides=1, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(10, 3, strides=1, activation="relu"))
model.add(layers.Conv2D(15, 3, strides=1, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(20, 3, strides=1, activation="relu"))
model.add(layers.Conv2D(20, 3, strides=1, activation="relu"))
model.add(layers.MaxPooling2D(3))


model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(500))
model.add(layers.Dense(50))
model.add(layers.Dense(10))
model.summary()

keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 3)       84        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 5)       140       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 5)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 10)      460       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 122, 122, 15)      1365      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 15)        0

In [7]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)


Epoch 1/50
21/21 [==============================] - 21s 992ms/step - loss: 8.3390 - accuracy: 0.1402 - val_loss: 11.7994 - val_accuracy: 0.1049
Epoch 2/50
21/21 [==============================] - 18s 863ms/step - loss: 10.9396 - accuracy: 0.1279 - val_loss: 11.6408 - val_accuracy: 0.1049
Epoch 3/50
21/21 [==============================] - 18s 844ms/step - loss: 11.3249 - accuracy: 0.1279 - val_loss: 11.7403 - val_accuracy: 0.1049
Epoch 4/50
21/21 [==============================] - 20s 940ms/step - loss: 11.8961 - accuracy: 0.1279 - val_loss: 11.7403 - val_accuracy: 0.1049
Epoch 5/50
21/21 [==============================] - 20s 938ms/step - loss: 11.9209 - accuracy: 0.1279 - val_loss: 11.7403 - val_accuracy: 0.1049
Epoch 6/50
21/21 [==============================] - 18s 862ms/step - loss: 11.9209 - accuracy: 0.1279 - val_loss: 11.7403 - val_accuracy: 0.1049
Epoch 7/50
11/21 [==============>...............] - ETA: 8s - loss: 11.4933 - accuracy: 0.1051

KeyboardInterrupt: 